In [48]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
import json

In [49]:
json_file = open("API_key.json")
API_KEY = json.load(json_file)["API_key"]
json_file.close()

REGION = 'euw1'
REGION_EU = 'europe'
watcher = LolWatcher(API_KEY)

### Get ID of a user

In [50]:
def get_id(summoner_name, region='euw1'):
    player_id = watcher.summoner.by_name(region, summoner_name)["id"]
    return player_id

### Get puuid of a user

In [51]:
def get_puuid(summoner_name, region='euw1'):
    puuid = watcher.summoner.by_name(region, summoner_name)["puuid"]
    return puuid

### Get item name from ID

In [52]:
def get_item_name(item_id):
    item_version = watcher.data_dragon.versions_for_region(REGION)['n']['item']
    item_name = watcher.data_dragon.items(item_version)['data'][str(item_id)]['name']
    return item_name

### Generate 1 row of data for specific champion from the history of a user

In [53]:
player_puuid = get_puuid("Pied à coulisse")
my_matches = watcher.match.matchlist_by_puuid(REGION_EU,player_puuid)

In [54]:
row = []
champ_list = []
item_list = []

target_champ = 'Sett'
matches_number = 5 #number of matches to analyze to found data

for L in range(matches_number):

    for i in range(10): 
        test_match = watcher.match.by_id(REGION_EU, my_matches[L])
        champion = test_match['info']['participants'][i]['championName']
        champ_list = np.append(champ_list,champion)
    
    if target_champ in champ_list:
        target_champ_idx = np.where(champ_list == target_champ)[0][0]
        row = champ_list
        for k in range(6):
                try:
                    item_n_str = 'item'+str(k)
                    item = get_item_name(test_match['info']['participants'][target_champ_idx][item_n_str])
                    row = np.append(row,item)
        
                except:
                    row = np.append(row,'no item')
        
    champ_list = []
    

print(row)

[]
